# `public_holyday_scraper`

creates a file called `public_holydays.ics` in the same directory. I use it to import the public holydays in my private protonmail calendar.
hard-coded for the canton of zurich

In [7]:
!pip install requests beautifulsoup4 ics

# imports

In [68]:
from ics import Calendar, Event
from bs4 import BeautifulSoup
import requests
from typing import Optional

import locale
from datetime import datetime, timedelta

# lib

In [121]:
locale.setlocale(locale.LC_ALL, 'de_CH')

def occurance_to_date(date:str, year:str) -> datetime:
    return datetime.strptime(f"{date} {year}", '%d. %B %Y').replace(hour=0, minute=0, tzinfo=None)

def row_to_dates(row:str, first_year, second_year):
    name, first_year_occurance, second_year_occurance = [c.text.strip() for c in row.children if c != '\n']
    
    if name:
        return (name, occurance_to_date(first_year_occurance, first_year)), (name, occurance_to_date(second_year_occurance, second_year))
    else:
        None

def flatten(ls):
    return [item for sub in ls for item in sub] 

# execution

In [15]:
ZURICH_URL = 'https://www.zh.ch/de/wirtschaft-arbeit/arbeitsbedingungen/arbeitsssicherheit-gesundheitsschutz/arbeits-ruhezeiten/feiertage.html'
raw_data  = BeautifulSoup(requests.get(ZURICH_URL).text, 'html.parser')

In [20]:
all_rows = raw_data.find_all(class_="mdl-table__row")

In [60]:
first_year, second_year = [t.text for t in all_rows[0].find_all('th') if t.text != '\xa0']

In [119]:
calendar = Calendar()

for name, day in flatten(filter(lambda x: x is not None, [row_to_dates(row, first_year, second_year) for row in all_rows[1:]])):
    e = Event(name=name, begin=day)
    e.make_all_day()
    calendar.events.add(e)

open('public_holydays.ics', 'w').writelines(calendar)